# Sparse Embeddings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
from datasets import load_dataset
from qdrant_client import models, QdrantClient
from fastembed import SparseTextEmbedding

from src import load_data, embeddings


c:\Users\manua\Documents\repos\dslabs\dslab-rag-retrieval\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Params

In [3]:
INDEX = "ragquas-sparse-01"
RETRIEVE_K=5

# Environment variables

In [4]:
load_dotenv()


True

# Clients

In [5]:
client_qdrant = QdrantClient(":memory:") 


# Load Data

In [6]:
df_ragquas = load_data.ragquas()
df_ragquas.head(2)

,topic,answer,question,variant,context_1,context_2,context_3,context_4,context_5,link_1,link_2,link_3,link_4,link_5,text_1,text_2,text_3,text_4,text_5
0,reclamaciones,La opción más fácil y eficaz para reclamar una...,¿Cuál es la forma más fácil de reclamar cuando...,question_1,#1. Airhelp\n\nLa empresa Airhelp fue fundada ...,En AirHelp hemos ayudado a más de 16 millones ...,"MYFLYRIGHT, expertos en derechos de los viajer...",,,https://www.businessinsider.es/mejores-paginas...,https://www.airhelp.com/es/retrasos-de-vuelos/,https://myflyright.com/es/servicios/vuelo-retr...,,,5 páginas donde poder reclamar tus vuelos por ...,Conoce las compensaciones cuando se retrasa tu...,Indemnización retraso vuelo\n\nNavegación por ...,,
1,reclamaciones,La opción más fácil y eficaz para reclamar una...,hola mi vuelo a Alemania se ha retrasado mucho...,question_2,#1. Airhelp\n\nLa empresa Airhelp fue fundada ...,En AirHelp hemos ayudado a más de 16 millones ...,"MYFLYRIGHT, expertos en derechos de los viajer...",,,https://www.businessinsider.es/mejores-paginas...,https://www.airhelp.com/es/retrasos-de-vuelos/,https://myflyright.com/es/servicios/vuelo-retr...,,,5 páginas donde poder reclamar tus vuelos por ...,Conoce las compensaciones cuando se retrasa tu...,Indemnización retraso vuelo\n\nNavegación por ...,,


In [7]:
df_docs = load_data.prepare_ragquas(df_ragquas)
df_docs.info()

Raw shape: (250, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     183 non-null    int64 
 1   topic     183 non-null    object
 2   variant   183 non-null    object
 3   question  183 non-null    object
 4   answer    183 non-null    object
 5   j         183 non-null    int64 
 6   context_  183 non-null    object
 7   text_     183 non-null    object
 8   link_     183 non-null    object
 9   uuid      183 non-null    object
dtypes: int64(2), object(8)
memory usage: 14.4+ KB


# How Sparse Vectors Works 

In [8]:
documents = [
    "You should stay, study and sprint.",
    "History can only prepare us to be surprised yet again.",
]

encoder = SparseTextEmbedding(model_name="Qdrant/bm25")
sp_embeddings = list(encoder.embed(documents))


print(f"{sp_embeddings[0].values=}")
print(f"{sp_embeddings[0].indices=}")

sp_embeddings[0].values=array([1.67868852, 1.67868852, 1.67868852])
sp_embeddings[0].indices=array([1881538586,  150760872, 1932363795])


In [9]:
type(sp_embeddings[0])

fastembed.sparse.sparse_embedding_base.SparseEmbedding

# Index

In [10]:
lst_embeddings = []

corpus_encoded = list(encoder.embed(df_docs['text_']))

for idx in range(len(corpus_encoded)):
    sp_vec = corpus_encoded[idx]
    lst_embeddings.append({
        "sparse": models.SparseVector(
            indices=sp_vec.indices,
            values=sp_vec.values
            )
        }
    )

print(f"{len(lst_embeddings)=}")

len(lst_embeddings)=183


In [11]:
lst_pts = embeddings.create_index_points(
    lst_embeddings,
    df_docs,
)
print(f"{len(lst_pts)=}")

len(lst_pts)=183


In [12]:
lst_qdrant_pts = embeddings.convert_to_qdrant_points(lst_pts)
print(f"{len(lst_qdrant_pts)=}")

len(lst_qdrant_pts)=183


In [13]:
client_qdrant.create_collection(
    collection_name=INDEX,
    vectors_config={},
    sparse_vectors_config={
        "sparse": models.SparseVectorParams(),
        }
)

client_qdrant.upsert(
    collection_name=INDEX,
    points=lst_qdrant_pts
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [14]:
_q_text = df_docs.iloc[0:1]['question']

list(encoder.embed(documents))[0]


SparseEmbedding(values=array([1.67868852, 1.67868852, 1.67868852]), indices=array([1881538586,  150760872, 1932363795]))

In [15]:
_q_text = df_docs.iloc[0:1]['question']

_q_encoded = next(encoder.embed(_q_text))



_q_vec = models.SparseVector(
    indices=_q_encoded.indices,
    values=_q_encoded.values
    )

print(f"Question: {_q_text}")
embeddings.check_query_qdrant(
        client_qdrant,
        INDEX,
        _q_vec,
        retrieve_k=3,
        using='sparse'  # passed by *args, **kwargs to query_points()
)



Question: 0    ¿Cuál es la forma más fácil de reclamar cuando...
Name: question, dtype: object
point.id='694cc886-48fe-42ca-a054-c7d470ee17df'
point.score=31.911108016967773
Doc: Conoce las compensaciones cuando se retrasa tu vuelo

Llegas al aeropuerto y tu vuelo va con retraso, ¿conoces cuáles son tus derechos y si puedes pedir reclamar una compensación? Desde AirHelp te ayudamos con los trámites para que tu viaje no se convierta en una pesadilla y puedas conocer todos los casos en los que puedes hacer una reclamación y el plazo para realizar los reclamos. ¡Te ayudaremos a reclamar tu indemnización por si tu vuelo se retrasa!

Desplazarse en avión es para muchas person...
idx: 0, j: 2
------------------------------
point.id='d6aedd59-ac3a-4c6b-8047-632efaee9d9a'
point.score=31.027894973754883
Doc: 5 páginas donde poder reclamar tus vuelos por retraso o cancelación de manera gratuita
Irina Pérez

    En muchas ocasiones no se realiza la reclamación de un vuelo por retraso o cancelació